<a href="https://colab.research.google.com/github/afroman32/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/wrangle-ml-datasets-assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inspect

In [0]:
import pandas as pd
import numpy as np

In [0]:
#need the encoding set to "ISO-8859-1" so it will read the weird characters properly 
movies = pd.read_csv('https://raw.githubusercontent.com/afroman32/Unit-2-Build/master/movies.csv',
                     encoding = "ISO-8859-1")

In [3]:
print(movies.shape)
movies.head()

(6820, 15)


,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [4]:
movies['score'].describe()

count    6820.000000
mean        6.374897
std         1.003142
min         1.500000
25%         5.800000
50%         6.400000
75%         7.100000
max         9.300000
Name: score, dtype: float64

#Wrangle

In [0]:
# director track record, movies success by director
# make sure to train on features that would come before the target

In [71]:
movies['director'].nunique

<bound method IndexOpsMixin.nunique of 0          Harley Cokeliss
1              Paul Bartel
2       John Carl Buechler
3          Sidney J. Furie
4            Bruno Bianchi
               ...        
6815         Nicole Garcia
6816       Eleanor Coppola
6817         Aisling Walsh
6818           Eric Summer
6819      Julian Rosefeldt
Name: director, Length: 6820, dtype: object>

In [54]:
movies['writer'].nunique()

4199

In [72]:
directors = list(movies['director'])
directors = list(dict.fromkeys(directors))
print(len(directors))
type(directors)

2759


list

In [61]:
sub = movies[movies['director'] == movies['director'][3]]
sub['gross'].sum()/len(sub['gross'])

13214797.4

In [82]:
ave_gross = []
for i in range(0,len(directors)+1):
  sub = movies[movies['director'] == movies['director'][i]]
  average=sub['gross'].sum()/len(sub['gross'])
  ave_gross.append(average)

# check is ave_gross has correct number of values
len(ave_gross)

2760

In [0]:
dir_ave = {directors[i]: ave_gross[i] for i in range(len(directors))}

In [117]:
movies['dir_ave_gross']= movies['director'].map(dir_ave)
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,net_revenue,dir_ave_gross
0,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,6637565.0,4849211.5
1,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1096843.0,1626657.0
2,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,4350815.0,12310408.0
3,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,24159872.0,13214797.4
4,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642.0,2193642.0


#Train, Val, Test Split 
(60,20,20)

In [0]:
movies.sort_values(by = 'released', inplace = True)

In [12]:
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,net_revenue
96,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,6637565.0
195,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1096843.0
47,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,4350815.0
27,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,24159872.0
218,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642.0


In [0]:
movies.reset_index(inplace = True, drop  = True)

In [14]:
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,net_revenue
0,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,6637565.0
1,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1096843.0
2,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,4350815.0
3,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,24159872.0
4,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642.0


In [0]:
# set end year for training data set
train_lim = movies['year'].min()+19

# set beginning year for test data
test_lim = movies['year'].max()-4

In [0]:
# movies in years less than training end year
train = movies[movies['year']<= train_lim]

# all movies in years greater than training end year
val_temp = movies[movies['year'] > train_lim]

# movies in years less than test year and greater than training end year
val = val_temp[val_temp['year'] < test_lim]

# movies in years greater than test year
test = movies[movies['year']>=test_lim]

In [17]:
train.shape, val.shape, test.shape

((4400, 16), (1320, 16), (1100, 16))

#Assignment 1

##Baseline

In [19]:
average = movies['gross'].mean()
print(f'{average:,.0f}')

33,497,829


In [20]:
from sklearn.metrics import mean_absolute_error

train['gross'].mean()

y_train = train['gross']
y_test = test['gross']
base_guess = y_train.mean()


# test error
y_base_pred = [base_guess] * len(y_test)
mae_test = mean_absolute_error(y_test, y_base_pred)

print(f'Mean Absolute Error: ${mae_test:,.2f}')

Mean Absolute Error: $45,726,662.13


##Basic Model

In [0]:
# set target
target = 'gross'

# set features
features = ['budget', 'genre', 'rating', 'year']

# set X matrices and y vectors
X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

In [22]:
!pip install category_encoders==2.*

     |████████████████████████████████| 102kB 3.7MB/s 


In [23]:
X_train.head()

,budget,genre,rating,year
0,0.0,Action,R,1986
1,0.0,Comedy,PG-13,1986
2,1100000.0,Comedy,PG-13,1986
3,0.0,Action,PG-13,1986
4,0.0,Family,G,1986


In [24]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
# from math import sqrt

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    # SimpleImputer(strategy = 'mean'),
    StandardScaler(),
    LinearRegression()
)

# Fit on training set
pipeline.fit(X_train, y_train)

# make predictions
y_pred = pipeline.predict(X_test)

# test mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error: {mae:,.2f}')

Test Mean Absolute Error: 28,700,487.17
